In [1]:
import numpy as np
import torch

In [2]:
def gen_data(bias=0, N=500):
    X = np.linspace(bias, bias + 5, N).reshape(-1, 1)
    mu = np.exp(-X + bias)
    eps = np.random.normal(0, 1, X.shape)
    sigma = 0.05 * (X - bias + 0.5)
    return X, mu + eps * sigma

In [3]:
X, y = gen_data(0, 500)
X = torch.from_numpy(X).to(torch.float32)
y = torch.from_numpy(y).to(torch.float32)

In [4]:
import sys
import os

cwd = os.getcwd()
h, _ = os.path.split(cwd)

sys.path.append(h)

In [5]:
from probaforms.models import DDPMUnconditional
from probaforms.models import DDPMConditional
from probaforms.models import DiffusionMLP
from probaforms.models import PlainBackboneResidual

## Conditional

In [6]:
backbone_args_dict = {
    'var_dim': 1,
    'cond_dim': 8,
    'hid_dim': 1024,
    'num_blocks': 6,
    'n_steps': 100,
    'steps_dim': 512,
    'steps_depth': 5,
    'steps_out_dim': 8,
}

backbone = PlainBackboneResidual(**backbone_args_dict)

In [7]:
beta_grid = 'linear'
betas = (1e-3, 1e-2) # min, max
sigma_method = 'beta_wave' # or "beta"

dmlp = DiffusionMLP(backbone, betas, beta_grid, sigma_method)
dmlp = dmlp.to('cuda')

In [8]:
start_lr = 1e-1
final_lr = 1e-2
n_epochs = 100
sched_lambda = (final_lr / start_lr) ** (1 / n_epochs)

optim = torch.optim.Adam(dmlp.parameters(), lr=start_lr, weight_decay=1e-5)
sched = torch.optim.lr_scheduler.ExponentialLR(optim, sched_lambda)
wrapper = DDPMConditional(dmlp, optim, n_epochs=n_epochs, batch_size=20, scheduler=None)

C:\Users\Oleg\anaconda3\envs\probaforms\lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [9]:
_ = wrapper.fit(y, X)
_ = wrapper.sample(X, batched=None).cpu()

batch: 100%|█████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.07it/s]


In [10]:
!pip install --upgrade torch

# Unconditional

In [11]:
backbone_args_dict = {
    'var_dim': 2,
    'cond_dim': None,
    'hid_dim': 1024,
    'num_blocks': 6,
    'n_steps': 100,
    'steps_dim': 512,
    'steps_depth': 5,
    'steps_out_dim': 8,
}

backbone = PlainBackboneResidual(**backbone_args_dict)

In [12]:
beta_grid = 'linear'
betas = (1e-3, 1e-2) # min, max
sigma_method = 'beta_wave' # or "beta"

dmlp = DiffusionMLP(backbone, betas, beta_grid, sigma_method)
dmlp = dmlp.to('cuda')

In [13]:
start_lr = 1e-1
final_lr = 1e-2
n_epochs = 100
sched_lambda = (final_lr / start_lr) ** (1 / n_epochs)

optim = torch.optim.Adam(dmlp.parameters(), lr=start_lr, weight_decay=1e-5)
sched = torch.optim.lr_scheduler.ExponentialLR(optim, sched_lambda)
wrapper = DDPMUnconditional(dmlp, optim, n_epochs=n_epochs, batch_size=20, scheduler=None)

In [14]:
data = torch.cat([X, y], dim=1)
_ = wrapper.fit(data)
_ = wrapper.sample(500).cpu()

batch: 100%|█████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.11it/s]
